Click the button above to load the interactive widgets

# SIR model

Try to fir the SIR model to the cases of a specific country and compare it to the logistic model

In [19]:
interact(SIRmodel_logbeta2, 
         timesteps=fixed(np.arange(0., 1000, 1)), 
         country = Dropdown(options=list(Confirmed.columns), value='US'),
         measured_cases = fixed(Confirmed),
         S_0=FloatSlider(value=500000, min=Confirmed.max().mean(), max=5*Confirmed.max().max(), step=1000,
                           continuous_update=False, description='Affected population'), I_0=fixed(1), R_0=fixed(0),
         log10_beta=FloatSlider(value=-6.3, min=-8, max=-5, step=0.001, continuous_update=True, readout_format='.2e',
                               description='log10(infection rate)'),
         gamma=FloatSlider(value=0.06, min=0, max=0.06, step=0.001, continuous_update=False, readout_format='.2e',
                          description='Recovery rate'), style = {'description_width': 'initial'},
#          layout=Layout(width='60%', height='300px'),
         returnDataFrame=fixed(False));

interactive(children=(FloatSlider(value=500000.0, continuous_update=False, description='Affected population', …

Below you can find the code

# Load modules

In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
# from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit, fsolve

from IPython.display import Image
from ipywidgets import interact, fixed, ToggleButtons, IntSlider, FloatSlider, Dropdown, SelectMultiple, Layout

# from pandas.tseries.offsets import Week
%matplotlib inline

<b> Preface </b>
    
    Here are being condensed few simple models and data visualization tools that capture my attention in literature. The idea is to play around with it and understand how important is to limit social contact in this period, and also the importance of modelling for understanding and timely actions.

# Load data from JHU

In [2]:
def loadData(fileName):
    data = pd.read_csv(baseURL + fileName) \
             .drop(['Lat', 'Long'], axis=1) #\
#              .melt(id_vars=['Province/State', 'Country/Region'], 
#                  var_name='date', value_name=columnName) \
#              .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
#                  errors='ignore')
    data['Province/State'].fillna('<all>', inplace=True)
#     data[columnName].fillna(0, inplace=True)
    return data

In [3]:
baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

In [4]:
Confirmed = loadData("time_series_covid19_confirmed_global.csv")

# Structure dataset

In [5]:
# grouping by country
Confirmed = Confirmed.groupby(['Country/Region']).sum()

In [6]:
Confirmed = Confirmed.T

In [7]:
Confirmed.index = pd.to_datetime(Confirmed.index)

# SIR Model definition

Inspired by https://www.linkedin.com/pulse/covid-19-from-model-prediction-predictive-control-ingmar-nopens/

In [8]:
def model_der(variables, t, beta, gamma):

    S = variables[0]
    I = variables[1]
    R = variables[2]
    
    S_new = -beta*I*S 
    I_new = beta*I*S - gamma*I
    R_new = gamma*I
    return [S_new, I_new, R_new]

In [9]:
def SIRmodel_logbeta2(timesteps, S_0, I_0, R_0, log10_beta, gamma, measured_cases, country, returnDataFrame=True, plotFig=True):
    """
    implementation of SIR population model
    
    Parameters
    -----------
    timesteps : np.array
    country: string
    log10_beta: float <0
    gamma: float
    
    """
    
    modeloutput = odeint(model_der, [S_0, I_0, R_0], timesteps, args=(10**(log10_beta), gamma));
    modeloutput = pd.DataFrame(modeloutput, columns=['S','I','R'], index=timesteps)
    
    modeloutput.index = pd.date_range(start='1/20/2020', periods=len(timesteps))
    
    if plotFig:
        fig, axes = plt.subplots(1,1, figsize=(10,8))
        axes.set_xlabel('time')
        axes.set_ylabel('Population')

        axes.plot(measured_cases[country], 'rx')
        axes.plot(modeloutput);
#         measured_cases[country].plot(axes=axes)
        
    if returnDataFrame:
        return modeloutput 